# Multimodal Article Question Answering Assistant

Code authored by: Shaw Talebi

### imports

In [1]:
import json
from functions import *
from transformers import CLIPProcessor, CLIPModel
from torch import load, matmul, argsort
from torch.nn.functional import softmax

from IPython.display import Image

import ollama

### load data

In [2]:
# load article contents
text_content_list = load_from_json('data/text_content.json')
image_content_list = load_from_json('data/image_content.json')

# load embeddings
text_embeddings = load('data/text_embeddings.pt', weights_only=True)
image_embeddings = load('data/image_embeddings.pt', weights_only=True)

In [3]:
print(text_embeddings.shape)
print(image_embeddings.shape)

torch.Size([86, 512])
torch.Size([17, 512])


In [4]:
text_content_list[49]

{'article_title': 'Multimodal Models\u200a—\u200aLLMs that can see and hear',
 'section': 'Path 1: LLM +\xa0Tools',
 'text': 'The key benefit of such an approach is simplicity. Tools can quickly be assembled without any additional model training.'}

### query

In [5]:
# query
query = "What is CLIP's contrastive loss function?"
# query = "What are the three paths described for making LLMs multimodal?"
# query = "What is an intuitive explanation of multimodal embeddings?"

# embed query
query_embed = embed_text(query)

In [6]:
query_embed.shape

torch.Size([1, 512])

### Multimodal search

In [7]:
k = 5
threshold = 0.05

# multimodal search over articles
text_similarities = matmul(query_embed, text_embeddings.T)
image_similarities = matmul(query_embed, image_embeddings.T)

# rescale similarities via softmax
temp=0.25
text_scores = softmax(text_similarities/temp, dim=1)
image_scores = softmax(image_similarities/temp, dim=1)

# return top k filtered text results
isorted_scores = argsort(text_scores, descending=True)[0]
sorted_scores = text_scores[0][isorted_scores]

itop_k_filtered = [idx.item() for idx, score in zip(isorted_scores, sorted_scores) if score.item() >= threshold][:k]
top_k = [text_content_list[i] for i in itop_k_filtered]

top_k

[{'article_title': 'Multimodal Models\u200a—\u200aLLMs that can see and hear',
  'section': 'What is a Multimodal Model?',
  'text': 'One benefit of using existing LLM as a starting point for MMs is that they’ve demonstrated a strong ability to acquire world knowledge through large-scale pre-training, which can be leveraged to process concepts appearing in non-textual representations.'},
 {'article_title': 'Multimodal Models\u200a—\u200aLLMs that can see and hear',
  'section': 'Multimodal Models\u200a—\u200aLLMs That Can See and\xa0Hear',
  'text': 'This has sparked efforts toward expanding LLM functionality to include multiple modalities.'},
 {'article_title': 'Multimodal Models\u200a—\u200aLLMs that can see and hear',
  'section': 'What is a Multimodal Model?',
  'text': 'While there are several ways to create models that can process multiple data modalities, a recent line of research seeks to use LLMs as the core reasoning engine of a multimodal system [2]. Such models are called m

#### text and image search

In [8]:
text_results, text_scores = similarity_search(query_embed, text_embeddings, text_content_list, k=15, threshold=0.01, temperature=0.25)
image_results, image_scores = similarity_search(query_embed, image_embeddings, image_content_list, k=5, threshold=0.25, temperature=0.5)

In [9]:
for text in text_results:
    if text_results:
        print("*", text['text'])

* One benefit of using existing LLM as a starting point for MMs is that they’ve demonstrated a strong ability to acquire world knowledge through large-scale pre-training, which can be leveraged to process concepts appearing in non-textual representations.
* This has sparked efforts toward expanding LLM functionality to include multiple modalities.
* While there are several ways to create models that can process multiple data modalities, a recent line of research seeks to use LLMs as the core reasoning engine of a multimodal system [2]. Such models are called multimodal large language models (or large multimodal models) [2][3].
* GPT-4o — Input: text, images, and audio. Output: text.FLUX — Input: text. Output: images.Suno — Input: text. Output: audio.
* The downside, however, is that the quality of such a system may be limited. Just like when playing a game of telephone, messages mutate when passed from person to person. Information may degrade going from one module to another via text 

In [10]:
for image in image_results:
    display(Image(filename=image['image_path']))

### Prompt Engineering

#### format context

In [11]:
text_context = ""
for text in text_results:
    if text_results:
        text_context = text_context + "**Article title:** " + text['article_title'] + "\n"
        text_context = text_context + "**Section:**  " + text['section'] + "\n"
        text_context = text_context + "**Snippet:** " + text['text'] + "\n\n"

In [12]:
image_context = ""
for image in image_results:
    if image_results:
        image_context = image_context + "**Article title:** " + image['article_title'] + "\n"
        image_context = image_context + "**Section:**  " + image['section'] + "\n"
        image_context = image_context + "**Image Path:**  " + image['image_path'] + "\n"
        image_context = image_context + "**Image Caption:** " + image['caption'] + "\n\n"

#### prompt template

In [13]:
# construct prompt template
prompt_template = lambda query, text_context, image_context : f"""Given the query "{query}" and the following relevant snippets:

{text_context}
{image_context}

Please provide a concise and accurate answer to the query, incorporating relevant information from the provided snippets where possible.

"""

In [14]:
print(prompt_template(query, text_results, image_results))

Given the query "What are the three paths described for making LLMs multimodal?" and the following relevant snippets:

**Article title:** Multimodal Models — LLMs that can see and hear
**Section:**  What is a Multimodal Model?
**Snippet:** One benefit of using existing LLM as a starting point for MMs is that they’ve demonstrated a strong ability to acquire world knowledge through large-scale pre-training, which can be leveraged to process concepts appearing in non-textual representations.

**Article title:** Multimodal Models — LLMs that can see and hear
**Section:**  Multimodal Models — LLMs That Can See and Hear
**Snippet:** This has sparked efforts toward expanding LLM functionality to include multiple modalities.

**Article title:** Multimodal Models — LLMs that can see and hear
**Section:**  What is a Multimodal Model?
**Snippet:** While there are several ways to create models that can process multiple data modalities, a recent line of research seeks to use LLMs as the core reason

### Prompt LLM

In [15]:
ollama.pull('llama3.2-vision')

ProgressResponse(status='success', completed=None, total=None, digest=None)

In [16]:
prompt = prompt_template(query, text_results, image_results)

response = ollama.chat(
    model='llama3.2-vision',
    messages=[{
        'role': 'user',
        'content': prompt,
        'images': [image["image_path"] for image in image_results]
    }]
)

print(response['message']['content'])

The three paths described for making Large Language Models (LLMs) multimodal are:

1. **LLM + Tools**: This approach involves augmenting an LLM with pre-built components that can process multiple data modalities. The key benefit is simplicity, as tools can be quickly assembled without additional model training. However, the quality of such a system may be limited due to information degradation when passing messages from one module to another via text descriptions only.

2. **LLM + Adapters**: This approach involves augmenting an LLM with multi-modal encoders or decoders that are aligned via adapter fine-tuning. Unfortunately, this approach is not explicitly mentioned in the provided snippets, but it can be inferred as a possible method for making LLMs multimodal.

3. **Unified Models**: This approach involves expanding the architecture of an LLM to fuse multiple modalities into a shared concept space at pre-training time. While this approach comes with greater technical challenges and 